<h1>ARIMA</h1>

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from numba import jit
import math

import pmdarima as pm

In [21]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [22]:
train_1_reduce = train_1[0:100]
train_1_reduce = train_1_reduce.fillna(0)
train_1_reduce = train_1_reduce.set_index('Page')
train_1_reduce = train_1_reduce.T.rename_axis('Dates')
train_1_reduce.head()

Page,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,Lovelyz_zh.wikipedia.org_all-access_spider,Lulu_zh.wikipedia.org_all-access_spider,MADTOWN_zh.wikipedia.org_all-access_spider,MAMAMOO_zh.wikipedia.org_all-access_spider,MONSTA_X_zh.wikipedia.org_all-access_spider,Madame_Antoine_zh.wikipedia.org_all-access_spider,Mamamoo_zh.wikipedia.org_all-access_spider,Mike_D._Angelo_zh.wikipedia.org_all-access_spider,Miss_A_zh.wikipedia.org_all-access_spider,Missing9_zh.wikipedia.org_all-access_spider
Dates,,,,,,,,,,,,,,,,,,,,,
2015-07-01,18.0,11.0,1.0,35.0,0.0,12.0,0.0,118.0,5.0,6.0,...,2.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,3.0,0.0
2015-07-02,11.0,14.0,0.0,13.0,0.0,7.0,0.0,26.0,23.0,3.0,...,6.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,9.0,0.0
2015-07-03,5.0,15.0,1.0,10.0,0.0,4.0,0.0,30.0,14.0,5.0,...,5.0,0.0,1.0,17.0,0.0,0.0,0.0,0.0,12.0,0.0
2015-07-04,13.0,18.0,1.0,94.0,0.0,5.0,0.0,24.0,12.0,12.0,...,21.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,14.0,0.0
2015-07-05,14.0,11.0,0.0,4.0,0.0,20.0,0.0,29.0,9.0,6.0,...,4.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,6.0,0.0


In [23]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [24]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE'])

train = train_1_reduce[:500]
test = train_1_reduce[500:550]
for column in train_1_reduce.columns:
    print(column)
    model = pm.auto_arima(train[column], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
    preds, conf = model.predict(n_periods=test.shape[0], return_conf_int=True) # renvoie prédiction et intervalles de confiances
    print(preds)
    predictions = pd.DataFrame(preds, index = test.index)
    print(predictions)
    lower_series = pd.Series(conf[:, 0], index=test.index)
    upper_series = pd.Series(conf[:, 1], index=test.index)
    rmse = np.sqrt(mean_squared_error(test[column], preds))
    smape = smape_fast(test[column].values, preds)
    print("Test RMSE: %.3f" % rmse)
    print("Test SMAPE: %.3f" % smape)
    metrics_df.loc[column] = [rmse,smape]

2NE1_zh.wikipedia.org_all-access_spider
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=4788.699, Time=0.11 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=5012.752, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=4866.723, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=4923.083, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=4776.894, Time=0.39 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=4835.207, Time=0.12 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=4775.352, Time=0.46 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=4824.954, Time=0.13 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=4776.396, Time=0.78 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=4773.441, Time=0.54 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=4780.951, Time=0.25 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=4778.434, Time=0.67 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=4777.175, Time=0.58 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=inf, Time=0.79 se

 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=3449.088, Time=0.56 sec

Best model:  ARIMA(1,0,1)(0,0,0)[0] intercept
Total fit time: 4.855 seconds
[2.2557249  3.53210933 4.18301907 4.51495941 4.68423693 4.77056232
 4.81458514 4.83703518 4.84848389 4.85432231 4.8572997  4.85881805
 4.85959236 4.85998723 4.8601886  4.86029129 4.86034366 4.86037036
 4.86038398 4.86039093 4.86039447 4.86039628 4.8603972  4.86039767
 4.86039791 4.86039803 4.86039809 4.86039812 4.86039814 4.86039815
 4.86039815 4.86039815 4.86039815 4.86039816 4.86039816 4.86039816
 4.86039816 4.86039816 4.86039816 4.86039816 4.86039816 4.86039816
 4.86039816 4.86039816 4.86039816 4.86039816 4.86039816 4.86039816
 4.86039816 4.86039816]
                   0
Dates               
2016-11-12  2.255725
2016-11-13  3.532109
2016-11-14  4.183019
2016-11-15  4.514959
2016-11-16  4.684237
2016-11-17  4.770562
2016-11-18  4.814585
2016-11-19  4.837035
2016-11-20  4.848484
2016-11-21  4.854322
2016-11-22  4.857300
2016-11-23  4.858818
201

 ARIMA(2,0,1)(0,0,0)[0]             : AIC=4030.910, Time=0.32 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=4137.549, Time=0.09 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=4032.819, Time=0.34 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=4032.721, Time=0.39 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=4030.724, Time=0.31 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=4252.540, Time=0.13 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=4032.722, Time=0.33 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=4215.335, Time=0.20 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=inf, Time=0.77 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=4028.625, Time=0.60 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=4056.250, Time=0.30 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=4035.761, Time=0.40 sec
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.83 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=4030.549, Time=0.82 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=4058.257, Time=0.18 sec
 ARIMA(0,0,3)(0,0,0)

 ARIMA(1,0,1)(0,0,0)[0]             : AIC=4379.256, Time=0.21 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=5094.542, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=4564.637, Time=0.07 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=4840.205, Time=0.09 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=4368.127, Time=0.34 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=4500.421, Time=0.13 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=4370.126, Time=0.38 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=4370.126, Time=0.45 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=4368.564, Time=0.33 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=4471.638, Time=0.18 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=inf, Time=0.50 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=4375.143, Time=0.73 sec

Best model:  ARIMA(2,0,1)(0,0,0)[0]          
Total fit time: 3.430 seconds
[39.31868043 35.84974503 35.25979119 35.1513849  35.12352646 35.10914386
 35.09701848 35.08527388 35.0735962  35.06193294 35.05027533 35

 ARIMA(0,0,1)(0,0,0)[0]             : AIC=4258.525, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=4043.706, Time=0.17 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=4043.636, Time=0.24 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=4205.329, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=4127.803, Time=0.12 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=4045.189, Time=0.49 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=4030.155, Time=0.42 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=4067.212, Time=0.17 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=4063.422, Time=0.13 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=4030.497, Time=0.50 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=4030.174, Time=0.36 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=4077.522, Time=0.02 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=4059.668, Time=0.32 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=4052.893, Time=0.29 sec
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=4028.710, Time=1.03 sec
 ARIMA(3,0

...

 ARIMA(2,0,1)(0,0,0)[0]             : AIC=3307.352, Time=0.24 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=3307.140, Time=0.35 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=3713.342, Time=0.09 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=3466.782, Time=0.05 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=3304.969, Time=0.42 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=3303.376, Time=0.63 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=3302.115, Time=0.25 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=3392.030, Time=0.13 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=3300.236, Time=0.85 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=3306.162, Time=0.69 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=3323.329, Time=0.30 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=3301.593, Time=0.94 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=3335.938, Time=0.15 sec
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=3303.149, Time=1.00 sec

Best model:  ARIMA(3,0,1)(0,0,0)[0] intercept
Total fit time: 6.425 seconds

 ARIMA(0,0,1)(0,0,0)[0]             : AIC=5400.373, Time=0.10 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=5174.795, Time=0.25 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=5251.559, Time=0.08 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=5175.468, Time=0.46 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=5170.023, Time=0.28 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=5175.739, Time=0.21 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=5169.029, Time=0.41 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=5169.167, Time=0.79 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=5169.745, Time=0.47 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=5166.950, Time=0.91 sec
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=5166.209, Time=0.67 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=5173.601, Time=0.68 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=5172.966, Time=0.65 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=5167.334, Time=0.93 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=5175.725, Time=0.42 sec
 ARIMA(1,0

 ARIMA(1,0,0)(0,0,0)[0]             : AIC=4663.707, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=4713.046, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=4506.010, Time=0.24 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=4506.010, Time=0.26 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=4669.253, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=4606.781, Time=0.11 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=4506.407, Time=0.36 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=4503.522, Time=0.55 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=4557.676, Time=0.26 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=4553.939, Time=0.08 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=4505.522, Time=0.71 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=4505.522, Time=0.77 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=4567.250, Time=0.02 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=4547.664, Time=0.24 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=4541.764, Time=0.24 sec
 ARIMA(2,0

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=3995.255, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=3791.984, Time=0.07 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=3880.406, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=3668.805, Time=0.24 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=3743.673, Time=0.10 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=3670.372, Time=0.34 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=3670.665, Time=0.35 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=3669.201, Time=0.23 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=3735.940, Time=0.08 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=3672.124, Time=0.48 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=3669.712, Time=0.38 sec

Best model:  ARIMA(2,0,1)(0,0,0)[0]          
Total fit time: 2.483 seconds
[22.73588907 24.35668352 24.46583406 24.41326552 24.34356456 24.27221696
 24.2008808  24.12973312 24.05879235 23.98805989 23.91753536 23.84721817
 23.77710771 23.70720337 23.63750455 23.56801065 23

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=3463.994, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=3373.199, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=3404.601, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=3290.492, Time=0.23 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=3356.900, Time=0.10 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=3291.248, Time=0.30 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=3293.235, Time=0.18 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=3290.214, Time=0.21 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=3375.490, Time=0.14 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=3292.178, Time=0.28 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=3370.719, Time=0.14 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=3261.377, Time=0.31 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=3235.466, Time=0.94 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=3282.608, Time=0.34 sec
 ARIMA(3,0,3)(0,0,0)[0] intercept   : AIC=3236.559, Time=1.23 sec

Best mode

In [25]:
metrics_df

,RMSE,SMAPE
2NE1_zh.wikipedia.org_all-access_spider,29.256888,41.338846
2PM_zh.wikipedia.org_all-access_spider,85.262569,43.573924
3C_zh.wikipedia.org_all-access_spider,29.152566,54.224639
4minute_zh.wikipedia.org_all-access_spider,8.382365,38.963237
52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,28.399120,93.524692
...,...,...
Madame_Antoine_zh.wikipedia.org_all-access_spider,3.089670,46.399092
Mamamoo_zh.wikipedia.org_all-access_spider,59.595126,60.946011
Mike_D._Angelo_zh.wikipedia.org_all-access_spider,8.606111,43.072589
Miss_A_zh.wikipedia.org_all-access_spider,5.386693,26.510518


In [26]:
moyenne_RMSE = metrics_df['RMSE'].mean()
somme_RMSE = metrics_df['RMSE'].sum()
moyenne_SMAPE = metrics_df['SMAPE'].mean()

In [27]:
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)

Test moyenne RMSE: 19.721
Test somme RMSE: 1972.085
Test moyenne SMAPE: 55.435
